In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

'02.08.2022'

In [25]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('FanDuel-NBA*.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [43]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [44]:
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position
0,71337-55062,C,Nikola,Nikola Jokic,Jokic,57.155320,47.0,11500,NY@DEN,DEN,NY,NaN,NaN,NaN,NaN,NaN,C
1,71337-40199,PF/SF,Giannis,Giannis Antetokounmpo,Antetokounmpo,55.033333,45.0,11300,MIL@LAL,MIL,LAL,NaN,NaN,NaN,NaN,NaN,SF/PF
2,71337-84669,PG/SG,Luka,Luka Doncic,Doncic,51.092307,39.0,11200,DET@DAL,DAL,DET,NaN,NaN,NaN,NaN,NaN,PG/SG
3,71337-49111,C,Joel,Joel Embiid,Embiid,53.260974,41.0,11000,PHO@PHI,PHI,PHO,NaN,NaN,NaN,NaN,NaN,C
4,71337-15755,PF/C,Anthony,Anthony Davis,Davis,48.196970,33.0,10800,MIL@LAL,LAL,MIL,GTD,Wrist,NaN,NaN,NaN,PF/C


In [45]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [46]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,Brandon Ingram,NOP,HOU,35.5,SF,"7,800",47.2,11.0,56.0,42.4,MAIN,Brandon Ingram,SF,71337-66048
1,Keifer Sykes,IND,ATL,33.0,PG,"3,500",28.1,9.1,37.7,42.4,MAIN,Keifer Sykes,PG,71337-18726
2,Lance Stephenson,IND,ATL,31.5,SF,"4,500",31.6,8.6,40.7,7.2,MAIN,Lance Stephenson,SF,71337-12369
3,Jonas Valanciunas,NOP,HOU,30.0,C,"7,200",42.2,8.4,50.5,10.4,MAIN,Jonas Valanciunas,C,71337-14503
4,Jusuf Nurkic,POR,ORL,32.5,C,"7,700",43.9,8.1,53.8,14.4,MAIN,Jusuf Nurkic,C,71337-55057


In [47]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position
0,Brandon Ingram,NOP,HOU,35.5,SF,"7,800",47.2,11.0,56.0,42.4,MAIN,Brandon Ingram,SF,71337-66048,,F
1,Keifer Sykes,IND,ATL,33.0,PG,"3,500",28.1,9.1,37.7,42.4,MAIN,Keifer Sykes,PG,71337-18726,G,
2,Lance Stephenson,IND,ATL,31.5,SF,"4,500",31.6,8.6,40.7,7.2,MAIN,Lance Stephenson,SF,71337-12369,,F
3,Jonas Valanciunas,NOP,HOU,30.0,C,"7,200",42.2,8.4,50.5,10.4,MAIN,Jonas Valanciunas,C,71337-14503,,
4,Jusuf Nurkic,POR,ORL,32.5,C,"7,700",43.9,8.1,53.8,14.4,MAIN,Jusuf Nurkic,C,71337-55057,,


In [48]:
df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Brandon Ingram,NOP,HOU,35.5,SF,"7,800",47.2,11.0,56.0,42.4,MAIN,Brandon Ingram,SF,71337-66048,NaN,F,SF/F
1,Keifer Sykes,IND,ATL,33.0,PG,"3,500",28.1,9.1,37.7,42.4,MAIN,Keifer Sykes,PG,71337-18726,G,NaN,PG/G
2,Lance Stephenson,IND,ATL,31.5,SF,"4,500",31.6,8.6,40.7,7.2,MAIN,Lance Stephenson,SF,71337-12369,NaN,F,SF/F
3,Jonas Valanciunas,NOP,HOU,30.0,C,"7,200",42.2,8.4,50.5,10.4,MAIN,Jonas Valanciunas,C,71337-14503,NaN,NaN,C
4,Jusuf Nurkic,POR,ORL,32.5,C,"7,700",43.9,8.1,53.8,14.4,MAIN,Jusuf Nurkic,C,71337-55057,NaN,NaN,C


In [49]:
# ## Unnests Roster Positions into each Row (explode in Python) ##


# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

# df['Roster Position'] = df['Position'].astype(str)

# df['Roster Position'] = df['Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Brandon Ingram,NOP,HOU,35.5,SF,7800.0,47.2,11.0,56.0,42.4,MAIN,Brandon Ingram,SF,71337-66048,NaN,F,SF/F
1,Keifer Sykes,IND,ATL,33.0,PG,3500.0,28.1,9.1,37.7,42.4,MAIN,Keifer Sykes,PG,71337-18726,G,NaN,PG/G
2,Lance Stephenson,IND,ATL,31.5,SF,4500.0,31.6,8.6,40.7,7.2,MAIN,Lance Stephenson,SF,71337-12369,NaN,F,SF/F
3,Jonas Valanciunas,NOP,HOU,30.0,C,7200.0,42.2,8.4,50.5,10.4,MAIN,Jonas Valanciunas,C,71337-14503,NaN,NaN,C
4,Jusuf Nurkic,POR,ORL,32.5,C,7700.0,43.9,8.1,53.8,14.4,MAIN,Jusuf Nurkic,C,71337-55057,NaN,NaN,C


In [50]:
df.to_excel("FD_final_PlayerPool_ETR_TEST"+str(today)+".xlsx", index=False)

df = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_TEST'+ str(today) + ".xlsx")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Brandon Ingram,NOP,HOU,35.5,SF,7800,47.2,11.0,56.0,42.4,MAIN,Brandon Ingram,SF,71337-66048,NaN,F,SF/F
1,Keifer Sykes,IND,ATL,33.0,PG,3500,28.1,9.1,37.7,42.4,MAIN,Keifer Sykes,PG,71337-18726,G,NaN,PG/G
2,Lance Stephenson,IND,ATL,31.5,SF,4500,31.6,8.6,40.7,7.2,MAIN,Lance Stephenson,SF,71337-12369,NaN,F,SF/F
3,Jonas Valanciunas,NOP,HOU,30.0,C,7200,42.2,8.4,50.5,10.4,MAIN,Jonas Valanciunas,C,71337-14503,NaN,NaN,C
4,Jusuf Nurkic,POR,ORL,32.5,C,7700,43.9,8.1,53.8,14.4,MAIN,Jusuf Nurkic,C,71337-55057,NaN,NaN,C


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [54]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [55]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [56]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

In [57]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['list'].iloc[i]) for i in range(len(df))])
    
# prob += get_position_sum(player_vars, df, 'PG') <= 2
prob += get_position_sum(player_vars, df, 'PG') >= 2
# prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 3
# prob += get_position_sum(player_vars, df, 'G') <= 4
prob += get_position_sum(player_vars, df, 'G') >= 4
# prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
# prob += get_position_sum(player_vars, df, 'PF') <= 2
prob += get_position_sum(player_vars, df, 'PF') >= 2
prob += get_position_sum(player_vars, df, 'F') >= 4
# prob += get_position_sum(player_vars, df, 'F') >= 2
# prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 2
# prob += get_position_sum(player_vars, df, 'UTIL') <= 2



In [58]:
# total salary constraint

prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

In [59]:
# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [60]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [61]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['list'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

SF/F Brandon Ingram NOP 7800 47.2
PG/G Keifer Sykes IND 3500 28.1
SF/F Lance Stephenson IND 4500 31.6
C Jonas Valanciunas NOP 7200 42.2
PG/SG/G Anfernee Simons POR 6500 36.5
PG/SG/G Luka Doncic DAL 11200 54.2
PF/F Jaxson Hayes NOP 5500 31.2
SF/PF/F Jayson Tatum BOS 9200 45.4
SG/G Torrey Craig IND 4600 25.9


(60000, 342.29999999999995)